ResNet with Attention

In [1]:
#load keras model
import keras
from keras.models import load_model
import numpy as np
#load model
model = load_model(r'C:\Users\Utente\Desktop\Chiara\Codici\Cose nuove\Nuovi\Final_mean\Codes\Trained models\ResNet_SE.keras')


In [2]:
import pandas as pd
test_sick = pd.read_pickle(r'C:\Users\Utente\Desktop\Chiara\SUBSET\df_malati_mean_noFMP.pkl')
test_healthy = pd.read_pickle(r'C:\Users\Utente\Desktop\Chiara\SUBSET\df_test_mean.pkl')
train = pd.read_pickle(r"C:\Users\Utente\Desktop\Chiara\SUBSET\df_train_mean.pkl")

In [3]:
test_healthy['signals'] = test_healthy['signals'].apply(lambda x: x[0] if isinstance(x, list) and isinstance(x[0], np.ndarray) else x)
test_sick['signals'] = test_sick['signals'].apply(lambda x: x[0] if isinstance(x, list) and isinstance(x[0], np.ndarray) else x)
train['signals'] = train['signals'].apply(lambda x: x[0] if isinstance(x, list) and isinstance(x[0], np.ndarray) else x)

In [4]:
def find_n_windows(l_signal, w_window=3600, Maxoverlap=0.5):
    n_windows = np.ceil(l_signal / w_window)
    if n_windows ==1:
        overlap = 0
    else:
        overlap = np.ceil(((n_windows * w_window) - l_signal) / (n_windows -1))
    if overlap * (n_windows - 1) > (w_window * Maxoverlap):
        n_windows = n_windows -1
        overlap = 0
    return n_windows, overlap


import numpy as np

def WindowSignals(X, y, Maxoverlap=0.5):
    """
    Divide i segnali di X in finestre e assegna a ciascuna finestra l'età gestazionale `y`.
    
    Args:
        X (ndarray): Array di segnali, ciascuno rappresentato come array numpy (dtype=object).
        y (ndarray): Array di etichette (età gestazionale) corrispondenti ai segnali in X (dtype=object).
        Maxoverlap (float): Percentuale massima di sovrapposizione tra le finestre (default 0.5).

    Returns:
        X_post_win (ndarray): Array di finestre dei segnali.
        y_post_win (ndarray): Array di etichette per ciascuna finestra.
    """
    if isinstance(X, pd.DataFrame) or isinstance(X, pd.Series):
        X = X.to_numpy()  # Conversione a numpy array se X è un DataFrame o Series
    if isinstance(y, pd.DataFrame) or isinstance(y, pd.Series):
        y = y.to_numpy()  # Conversione a numpy array se y è un DataFrame o Series
    

    X_post_win = []
    y_post_win = []

    for i in range(len(X)):  # Itera sugli array in X (ora sono ndarray di tipo object)
        signal = X[i]
        label = y[i]  # L'etichetta associata al segnale

        l_signal = len(signal)  # Lunghezza del segnale
        w_window = 3600  # Lunghezza della finestra

        # Calcola il numero di finestre e la sovrapposizione
        if Maxoverlap > 0:
            n_windows, overlap = find_n_windows(l_signal, w_window, Maxoverlap)
        else:
            n_windows = np.floor(l_signal / w_window)
            overlap = 0

        # Genera le finestre per il segnale corrente
        for n in range(int(n_windows)):
            start_index = int(n * (w_window - overlap))
            end_index = int(start_index + w_window)
            if end_index > l_signal:  # Verifica che la finestra non esca dai limiti
                break
            window = signal[start_index:end_index]
            X_post_win.append(window)
            y_post_win.append(label)  # Aggiungi l'etichetta corrispondente

    return np.array(X_post_win, dtype=object), np.array(y_post_win, dtype=object)


In [5]:
X_windows_train, y_windows_train= WindowSignals(train['signals'], train['y'])
X_windows_test, y_windows_test = WindowSignals(test_healthy['signals'], test_healthy['y'])
X_windows_test_sick, y_windows_test_sick = WindowSignals(test_sick['signals'], test_sick['y'])

In [6]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(y_windows_train.reshape(-1, 1))
y_windows_test_scaled = scaler.transform(y_windows_test.reshape(-1, 1))
y_windows_test_sick_scaled = scaler.transform(y_windows_test_sick.reshape(-1, 1))

In [7]:
y_windows_test = y_windows_test_scaled
y_windows_test_sick = y_windows_test_sick_scaled

In [8]:
print(X_windows_test.dtype)
print(X_windows_test_sick.dtype)


object
object


In [9]:
import tensorflow as tf

X_windows_test = tf.convert_to_tensor(X_windows_test, dtype=tf.float32)
X_windows_test_sick = tf.convert_to_tensor(X_windows_test_sick, dtype=tf.float32)
X_windows_test = tf.expand_dims(X_windows_test, axis=-1)
X_windows_test_sick = tf.expand_dims(X_windows_test_sick, axis=-1)

In [10]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr, spearmanr, linregress, f
import numpy as np

def calculate_p_value_from_r2(R_squared, n, k):
    # Calcola F-statistic
    F_stat = (R_squared / k) / ((1 - R_squared) / (n - k - 1))
    # Calcola p-value
    p_value = 1 - f.cdf(F_stat, dfn=k, dfd=n - k - 1)
    return p_value

def compute_metrics(y_true, y_pred):
    y_true_denorm = scaler.inverse_transform(y_true.reshape(-1, 1)).flatten()
    y_pred_denorm = scaler.inverse_transform(y_pred.reshape(-1, 1)).flatten()
    
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    mae_denorm = mean_absolute_error(y_true_denorm, y_pred_denorm)
    mse_denorm = mean_squared_error(y_true_denorm, y_pred_denorm)
    mae_denorm_div7 = mae_denorm / 7
    mse_denorm_div7 = mse_denorm / 7
    
    r2 = r2_score(y_true, y_pred)
    
    # Pearson & Spearman
    pearson_corr, p_value_pearson = pearsonr(y_true, y_pred)
    spearman_corr, p_value_spearman = spearmanr(y_true, y_pred)
    
    # Linear regression
    slope, intercept, r_value, p_linreg, std_err = linregress(y_true, y_pred)
    
    # P-value da R^2
    n = len(y_true)
    k = 1  # numero di variabili indipendenti
    p_value_r2 = calculate_p_value_from_r2(r2, n, k)
    
    return {
        'MAE_norm': mae,
        'MSE_norm': mse,
        'MAE_denorm': mae_denorm,
        'MSE_denorm': mse_denorm,
        'MAE_denorm_div7': mae_denorm_div7,
        'MSE_denorm_div7': mse_denorm_div7,
        'R2': r2,
        'P_value_R2': p_value_r2,
        'Pearson': pearson_corr,
        'Spearman': spearman_corr,
        'P_value_pearson': p_value_pearson,
        'P_value_spearman': p_value_spearman,
        'Slope': slope
    }



In [11]:
def bootstrap_metrics(y_true, y_pred, n_bootstrap=1000, random_state=None):
    np.random.seed(random_state)
    n = len(y_true)
    metrics_list = []

    for _ in range(n_bootstrap):
        idx = np.random.choice(range(n), size=n, replace=True)
        metrics = compute_metrics(y_true[idx], y_pred[idx])
        metrics_list.append(metrics)
    
    return metrics_list


In [12]:
def compute_confidence_intervals(metrics_list, alpha=0.05):
    keys = metrics_list[0].keys()
    ci_dict = {}
    for key in keys:
        values = [m[key] for m in metrics_list]
        lower = np.percentile(values, 100 * alpha / 2)
        upper = np.percentile(values, 100 * (1 - alpha / 2))
        mean = np.mean(values)
        ci_dict[key] = (mean, lower, upper)
    
    return ci_dict


In [13]:
y_true_healthy = y_windows_test.flatten()
y_pred_healthy = model.predict(X_windows_test).flatten()
metrics_boot_healthy = bootstrap_metrics(y_true_healthy, y_pred_healthy)
ci_healthy = compute_confidence_intervals(metrics_boot_healthy)

y_true_sick = y_windows_test_sick.flatten()
y_pred_sick = model.predict(X_windows_test_sick).flatten()
metrics_boot_sick = bootstrap_metrics(y_true_sick, y_pred_sick)
ci_sick = compute_confidence_intervals(metrics_boot_sick)


120/120 [==============================] - 6s 47ms/step


In [14]:
print("Healthy subjects:")
for metrica, (mean, lower, upper) in ci_healthy.items():
    print(f"{metrica}: {mean:.3f} (IC 95%: {lower:.3f} - {upper:.3f})")

print("\nSick subjects:")
for metrica, (mean, lower, upper) in ci_sick.items():
    print(f"{metrica}: {mean:.3f} (IC 95%: {lower:.3f} - {upper:.3f})")


Healthy subjects:
MAE_norm: 0.258 (IC 95%: 0.251 - 0.265)
MSE_norm: 0.106 (IC 95%: 0.102 - 0.111)
MAE_denorm: 8.137 (IC 95%: 7.919 - 8.352)
MSE_denorm: 105.530 (IC 95%: 100.777 - 110.619)
MAE_denorm_div7: 1.162 (IC 95%: 1.131 - 1.193)
MSE_denorm_div7: 15.076 (IC 95%: 14.397 - 15.803)
R2: 0.258 (IC 95%: 0.232 - 0.283)
P_value_R2: 0.000 (IC 95%: 0.000 - 0.000)
Pearson: 0.508 (IC 95%: 0.483 - 0.534)
Spearman: 0.500 (IC 95%: 0.472 - 0.525)
P_value_pearson: 0.000 (IC 95%: 0.000 - 0.000)
P_value_spearman: 0.000 (IC 95%: 0.000 - 0.000)
Slope: 0.253 (IC 95%: 0.240 - 0.267)

Sick subjects:
MAE_norm: 0.505 (IC 95%: 0.491 - 0.518)
MSE_norm: 0.452 (IC 95%: 0.429 - 0.474)
MAE_denorm: 15.901 (IC 95%: 15.464 - 16.324)
MSE_denorm: 448.303 (IC 95%: 425.539 - 470.745)
MAE_denorm_div7: 2.272 (IC 95%: 2.209 - 2.332)
MSE_denorm_div7: 64.043 (IC 95%: 60.791 - 67.249)
R2: 0.213 (IC 95%: 0.192 - 0.234)
P_value_R2: 0.000 (IC 95%: 0.000 - 0.000)
Pearson: 0.653 (IC 95%: 0.636 - 0.671)
Spearman: 0.661 (IC 95%: 0.

In [15]:
lower = np.percentile(y_true_healthy, 5)
upper = np.percentile(y_true_healthy, 95)
print(lower)
print(upper)

mask = (y_true_healthy >= lower) & (y_true_healthy <= upper)
#print(len(y_true), len(y_true_sick), len(y_pred), len(y_pred_sick))
mask_sick = (y_true_sick >= lower) & (y_true_sick <= upper)

filtered_y_true = np.array(y_true_healthy)[mask]
filtered_y_pred = np.array(y_pred_healthy)[mask]
filtered_y_true_sick = np.array(y_true_sick)[mask_sick]
filtered_y_pred_sick = np.array(y_pred_sick)[mask_sick]

-0.7142857142857135
0.5873015873015888


In [16]:
metrics_boot_healthy_filtered = bootstrap_metrics(filtered_y_true, filtered_y_pred)
ci_healthy_filtered = compute_confidence_intervals(metrics_boot_healthy_filtered)
metrics_boot_sick_filtered = bootstrap_metrics(filtered_y_true_sick, filtered_y_pred_sick)
ci_sick_filtered = compute_confidence_intervals(metrics_boot_sick_filtered)
print("Healthy subjects (filtered):")
for metrica, (mean, lower, upper) in ci_healthy_filtered.items():
    print(f"{metrica}: {mean:.3f} (IC 95%: {lower:.3f} - {upper:.3f})")
print("\nSick subjects (filtered):")
for metrica, (mean, lower, upper) in ci_sick_filtered.items():
    print(f"{metrica}: {mean:.3f} (IC 95%: {lower:.3f} - {upper:.3f})")

Healthy subjects (filtered):
MAE_norm: 0.227 (IC 95%: 0.221 - 0.233)
MSE_norm: 0.080 (IC 95%: 0.076 - 0.084)
MAE_denorm: 7.139 (IC 95%: 6.951 - 7.335)
MSE_denorm: 79.141 (IC 95%: 75.285 - 83.126)
MAE_denorm_div7: 1.020 (IC 95%: 0.993 - 1.048)
MSE_denorm_div7: 11.306 (IC 95%: 10.755 - 11.875)
R2: 0.190 (IC 95%: 0.157 - 0.222)
P_value_R2: 0.000 (IC 95%: 0.000 - 0.000)
Pearson: 0.454 (IC 95%: 0.425 - 0.482)
Spearman: 0.449 (IC 95%: 0.419 - 0.479)
P_value_pearson: 0.000 (IC 95%: 0.000 - 0.000)
P_value_spearman: 0.000 (IC 95%: 0.000 - 0.000)
Slope: 0.262 (IC 95%: 0.242 - 0.280)

Sick subjects (filtered):
MAE_norm: 0.256 (IC 95%: 0.249 - 0.263)
MSE_norm: 0.100 (IC 95%: 0.095 - 0.105)
MAE_denorm: 8.062 (IC 95%: 7.834 - 8.298)
MSE_denorm: 98.975 (IC 95%: 93.995 - 104.257)
MAE_denorm_div7: 1.152 (IC 95%: 1.119 - 1.185)
MSE_denorm_div7: 14.139 (IC 95%: 13.428 - 14.894)
R2: 0.225 (IC 95%: 0.192 - 0.257)
P_value_R2: 0.000 (IC 95%: 0.000 - 0.000)
Pearson: 0.482 (IC 95%: 0.452 - 0.511)
Spearman: 0.4

In [17]:
df_healthy = pd.DataFrame(ci_healthy, index=['Media', 'IC 2.5%', 'IC 97.5%']).T
print(df_healthy)
df_sick = pd.DataFrame(ci_sick, index=['Media', 'IC 2.5%', 'IC 97.5%']).T
print(df_sick)
df_healthy_filtered = pd.DataFrame(ci_healthy_filtered, index=['Media', 'IC 2.5%', 'IC 97.5%']).T
print(df_healthy_filtered)
df_sick_filtered = pd.DataFrame(ci_sick_filtered, index=['Media', 'IC 2.5%', 'IC 97.5%']).T
print(df_sick_filtered)

                          Media        IC 2.5%       IC 97.5%
MAE_norm           2.583247e-01   2.514077e-01   2.651508e-01
MSE_norm           1.063538e-01   1.015640e-01   1.114829e-01
MAE_denorm         8.137228e+00   7.919342e+00   8.352252e+00
MSE_denorm         1.055296e+02   1.007768e+02   1.106189e+02
MAE_denorm_div7    1.162461e+00   1.131335e+00   1.193179e+00
MSE_denorm_div7    1.507566e+01   1.439669e+01   1.580270e+01
R2                 2.575953e-01   2.322066e-01   2.832065e-01
P_value_R2         1.110223e-16   1.110223e-16   1.110223e-16
Pearson            5.081922e-01   4.828491e-01   5.342196e-01
Spearman           4.995838e-01   4.724776e-01   5.254016e-01
P_value_pearson   1.835159e-191  1.252420e-257  5.677701e-204
P_value_spearman  2.290950e-185  9.406001e-248  3.287788e-194
Slope              2.534580e-01   2.397450e-01   2.673294e-01
                         Media       IC 2.5%      IC 97.5%
MAE_norm          5.047979e-01  4.909336e-01  5.182290e-01
MSE_norm      

Inception Net

In [18]:
model_inc = load_model(r'C:\Users\Utente\Desktop\Chiara\Codici\Cose nuove\Nuovi\Final_mean\Codes\Trained models\Inception_ResNet_v1.keras')

In [19]:
#y_true_healthy = y_windows_test.flatten()
y_pred_healthy_inc = model_inc.predict(X_windows_test).flatten()
metrics_boot_healthy_inc = bootstrap_metrics(y_true_healthy, y_pred_healthy_inc)
ci_healthy_inc = compute_confidence_intervals(metrics_boot_healthy_inc)

#y_true_sick = y_windows_test_sick.flatten()
y_pred_sick_inc = model_inc.predict(X_windows_test_sick).flatten()
metrics_boot_sick_inc = bootstrap_metrics(y_true_sick, y_pred_sick_inc)
ci_sick_inc = compute_confidence_intervals(metrics_boot_sick_inc)

120/120 [==============================] - 9s 79ms/step


In [20]:
print("Healthy subjects:")
for metrica, (mean, lower, upper) in ci_healthy_inc.items():
    print(f"{metrica}: {mean:.3f} (IC 95%: {lower:.3f} - {upper:.3f})")

print("\nSick subjects:")
for metrica, (mean, lower, upper) in ci_sick_inc.items():
    print(f"{metrica}: {mean:.3f} (IC 95%: {lower:.3f} - {upper:.3f})")


Healthy subjects:
MAE_norm: 0.301 (IC 95%: 0.293 - 0.309)
MSE_norm: 0.144 (IC 95%: 0.137 - 0.151)
MAE_denorm: 9.479 (IC 95%: 9.238 - 9.733)
MSE_denorm: 142.650 (IC 95%: 136.307 - 149.384)
MAE_denorm_div7: 1.354 (IC 95%: 1.320 - 1.390)
MSE_denorm_div7: 20.379 (IC 95%: 19.472 - 21.341)
R2: -0.004 (IC 95%: -0.010 - -0.001)
P_value_R2: 0.995 (IC 95%: 1.000 - 1.000)
Pearson: 0.021 (IC 95%: -0.012 - 0.055)
Spearman: 0.025 (IC 95%: -0.010 - 0.058)
P_value_pearson: 0.301 (IC 95%: 0.001 - 0.933)
P_value_spearman: 0.251 (IC 95%: 0.001 - 0.920)
Slope: 0.000 (IC 95%: -0.000 - 0.001)

Sick subjects:
MAE_norm: 0.625 (IC 95%: 0.609 - 0.641)
MSE_norm: 0.670 (IC 95%: 0.640 - 0.701)
MAE_denorm: 19.687 (IC 95%: 19.183 - 20.201)
MSE_denorm: 664.616 (IC 95%: 635.414 - 695.910)
MAE_denorm_div7: 2.812 (IC 95%: 2.740 - 2.886)
MSE_denorm_div7: 94.945 (IC 95%: 90.773 - 99.416)
R2: -0.166 (IC 95%: -0.191 - -0.144)
P_value_R2: 1.000 (IC 95%: 1.000 - 1.000)
Pearson: 0.108 (IC 95%: 0.074 - 0.142)
Spearman: 0.075 (I

In [21]:
#filtered_y_true = np.array(y_true_healthy)[mask]
filtered_y_pred_inc = np.array(y_pred_healthy_inc)[mask]
#filtered_y_true_sick = np.array(y_true_sick)[mask_sick]
filtered_y_pred_sick_inc = np.array(y_pred_sick_inc)[mask_sick]

In [22]:
metrics_boot_healthy_filtered_inc = bootstrap_metrics(filtered_y_true, filtered_y_pred_inc)
ci_healthy_filtered_inc = compute_confidence_intervals(metrics_boot_healthy_filtered_inc)
metrics_boot_sick_filtered_inc = bootstrap_metrics(filtered_y_true_sick, filtered_y_pred_sick_inc)
ci_sick_filtered_inc = compute_confidence_intervals(metrics_boot_sick_filtered_inc)
print("Healthy subjects (filtered):")
for metrica, (mean, lower, upper) in ci_healthy_filtered_inc.items():
    print(f"{metrica}: {mean:.3f} (IC 95%: {lower:.3f} - {upper:.3f})")
print("\nSick subjects (filtered):")
for metrica, (mean, lower, upper) in ci_sick_filtered_inc.items():
    print(f"{metrica}: {mean:.3f} (IC 95%: {lower:.3f} - {upper:.3f})")

Healthy subjects (filtered):
MAE_norm: 0.256 (IC 95%: 0.250 - 0.262)
MSE_norm: 0.099 (IC 95%: 0.095 - 0.103)
MAE_denorm: 8.051 (IC 95%: 7.867 - 8.242)
MSE_denorm: 98.054 (IC 95%: 94.071 - 101.983)
MAE_denorm_div7: 1.150 (IC 95%: 1.124 - 1.177)
MSE_denorm_div7: 14.008 (IC 95%: 13.439 - 14.569)
R2: -0.004 (IC 95%: -0.009 - 0.000)
P_value_R2: 0.984 (IC 95%: 0.798 - 1.000)
Pearson: 0.049 (IC 95%: 0.015 - 0.084)
Spearman: 0.048 (IC 95%: 0.010 - 0.084)
P_value_pearson: 0.047 (IC 95%: 0.000 - 0.411)
P_value_spearman: 0.056 (IC 95%: 0.000 - 0.564)
Slope: 0.001 (IC 95%: 0.000 - 0.001)

Sick subjects (filtered):
MAE_norm: 0.302 (IC 95%: 0.295 - 0.310)
MSE_norm: 0.129 (IC 95%: 0.124 - 0.134)
MAE_denorm: 9.527 (IC 95%: 9.293 - 9.779)
MSE_denorm: 127.767 (IC 95%: 122.625 - 133.298)
MAE_denorm_div7: 1.361 (IC 95%: 1.328 - 1.397)
MSE_denorm_div7: 18.252 (IC 95%: 17.518 - 19.043)
R2: 0.000 (IC 95%: -0.002 - 0.002)
P_value_R2: 0.424 (IC 95%: 0.039 - 1.000)
Pearson: 0.053 (IC 95%: 0.011 - 0.096)
Spearma

In [23]:
df_healthy_inc = pd.DataFrame(ci_healthy_inc, index=['Media', 'IC 2.5%', 'IC 97.5%']).T
print(df_healthy_inc)
df_sick_inc = pd.DataFrame(ci_sick_inc, index=['Media', 'IC 2.5%', 'IC 97.5%']).T
print(df_sick_inc)
df_healthy_filtered_inc = pd.DataFrame(ci_healthy_filtered_inc, index=['Media', 'IC 2.5%', 'IC 97.5%']).T
print(df_healthy_filtered_inc)
df_sick_filtered_inc = pd.DataFrame(ci_sick_filtered_inc, index=['Media', 'IC 2.5%', 'IC 97.5%']).T
print(df_sick_filtered_inc)

                       Media     IC 2.5%    IC 97.5%
MAE_norm            0.300913    0.293267    0.308978
MSE_norm            0.143764    0.137372    0.150550
MAE_denorm          9.478764    9.237918    9.732820
MSE_denorm        142.649708  136.307368  149.383522
MAE_denorm_div7     1.354109    1.319703    1.390403
MSE_denorm_div7    20.378530   19.472481   21.340503
R2                 -0.004391   -0.009601   -0.000616
P_value_R2          0.994685    1.000000    1.000000
Pearson             0.021430   -0.012102    0.055349
Spearman            0.024804   -0.009880    0.057657
P_value_pearson     0.300894    0.001052    0.932785
P_value_spearman    0.250993    0.000642    0.919679
Slope               0.000239   -0.000136    0.000621
                       Media       IC 2.5%    IC 97.5%
MAE_norm            0.624969  6.089987e-01    0.641307
MSE_norm            0.669807  6.403770e-01    0.701345
MAE_denorm         19.686519  1.918346e+01   20.201179
MSE_denorm        664.616195  6.354140

ResNet with activity state

In [24]:
model_qa = load_model(r'C:\Users\Utente\Desktop\Chiara\Codici\Cose nuove\Nuovi\Final_mean\Codes\Trained models\model_with_QA_mean.keras')

In [25]:
train_q_a = pd.read_pickle(r"C:\Users\Utente\Desktop\Chiara\DockerImage\softmax_predictions_train_non_norm.pkl")
test_q_a = pd.read_pickle(r"C:\Users\Utente\Desktop\Chiara\DockerImage\softmax_predictions_test_non_norm.pkl")
test_sick_q_a = pd.read_pickle(r"C:\Users\Utente\Desktop\Chiara\DockerImage\softmax_predictions_test_sick_non_norm.pkl")

In [26]:
train = pd.concat([train, train_q_a], axis=1)
test = pd.concat([test_healthy, test_q_a], axis=1)
test_sick = pd.concat([test_sick, test_sick_q_a], axis=1)

In [27]:
X_windows_train, y_windows_train= WindowSignals(train['signals'], train['y'])
X_windows_test, y_windows_test = WindowSignals(test['signals'], test['y'])
X_windows_test_sick, y_windows_test_sick = WindowSignals(test_sick['signals'], test_sick['y'])

X_windows_train_qa, y_windows_train= WindowSignals(train['softmax_prediction_list'], train['y'])
X_windows_test_qa, y_windows_test_qa = WindowSignals(test['softmax_prediction_list'], test['y'])
X_windows_test_sick_qa, y_windows_test_sick_qa = WindowSignals(test_sick['softmax_prediction_list'], test_sick['y'])

In [28]:
y_windows_test_scaled = scaler.transform(y_windows_test.reshape(-1, 1))
y_windows_test_sick_scaled = scaler.transform(y_windows_test_sick.reshape(-1, 1))
y_windows_test = y_windows_test_scaled
y_windows_test_sick = y_windows_test_sick_scaled

In [29]:
X_test = np.stack([X_windows_test, X_windows_test_qa], axis=-1)  # (batch_size, timesteps, 2)
X_test_sick = np.stack([X_windows_test_sick, X_windows_test_sick_qa], axis=-1)  # (batch_size, timesteps, 2)

In [30]:
X_windows_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
X_windows_test_sick = tf.convert_to_tensor(X_test_sick, dtype=tf.float32)

In [31]:
y_windows_test = tf.convert_to_tensor(y_windows_test, dtype=tf.float32)
y_windows_test_sick = tf.convert_to_tensor(y_windows_test_sick, dtype=tf.float32)

In [32]:
#y_true_healthy = y_windows_test.flatten()
y_pred_healthy_qa = model_qa.predict(X_windows_test).flatten()
metrics_boot_healthy_qa = bootstrap_metrics(y_true_healthy, y_pred_healthy_qa)
ci_healthy_qa = compute_confidence_intervals(metrics_boot_healthy_qa)

#y_true_sick = y_windows_test_sick.flatten()
y_pred_sick_qa = model_qa.predict(X_windows_test_sick).flatten()
metrics_boot_sick_qa = bootstrap_metrics(y_true_sick, y_pred_sick_qa)
ci_sick_qa = compute_confidence_intervals(metrics_boot_sick_qa)

120/120 [==============================] - 6s 47ms/step


In [33]:
print("Healthy subjects:")
for metrica, (mean, lower, upper) in ci_healthy_qa.items():
    print(f"{metrica}: {mean:.3f} (IC 95%: {lower:.3f} - {upper:.3f})")

print("\nSick subjects:")
for metrica, (mean, lower, upper) in ci_sick_qa.items():
    print(f"{metrica}: {mean:.3f} (IC 95%: {lower:.3f} - {upper:.3f})")


Healthy subjects:
MAE_norm: 0.257 (IC 95%: 0.251 - 0.264)
MSE_norm: 0.106 (IC 95%: 0.101 - 0.110)
MAE_denorm: 8.096 (IC 95%: 7.894 - 8.305)
MSE_denorm: 104.683 (IC 95%: 99.723 - 109.592)
MAE_denorm_div7: 1.157 (IC 95%: 1.128 - 1.186)
MSE_denorm_div7: 14.955 (IC 95%: 14.246 - 15.656)
R2: 0.263 (IC 95%: 0.241 - 0.286)
P_value_R2: 0.000 (IC 95%: 0.000 - 0.000)
Pearson: 0.519 (IC 95%: 0.495 - 0.544)
Spearman: 0.516 (IC 95%: 0.491 - 0.542)
P_value_pearson: 0.000 (IC 95%: 0.000 - 0.000)
P_value_spearman: 0.000 (IC 95%: 0.000 - 0.000)
Slope: 0.228 (IC 95%: 0.216 - 0.242)

Sick subjects:
MAE_norm: 0.516 (IC 95%: 0.503 - 0.531)
MSE_norm: 0.466 (IC 95%: 0.445 - 0.489)
MAE_denorm: 16.266 (IC 95%: 15.842 - 16.711)
MSE_denorm: 462.869 (IC 95%: 441.090 - 485.368)
MAE_denorm_div7: 2.324 (IC 95%: 2.263 - 2.387)
MSE_denorm_div7: 66.124 (IC 95%: 63.013 - 69.338)
R2: 0.188 (IC 95%: 0.168 - 0.207)
P_value_R2: 0.000 (IC 95%: 0.000 - 0.000)
Pearson: 0.663 (IC 95%: 0.646 - 0.681)
Spearman: 0.666 (IC 95%: 0.6

In [34]:
#filtered_y_true = np.array(y_true_healthy)[mask]
filtered_y_pred_qa = np.array(y_pred_healthy_qa)[mask]
#filtered_y_true_sick = np.array(y_true_sick)[mask_sick]
filtered_y_pred_sick_qa = np.array(y_pred_sick_qa)[mask_sick]

In [35]:
metrics_boot_healthy_filtered_qa = bootstrap_metrics(filtered_y_true, filtered_y_pred_qa)
ci_healthy_filtered_qa = compute_confidence_intervals(metrics_boot_healthy_filtered_qa)
metrics_boot_sick_filtered_qa = bootstrap_metrics(filtered_y_true_sick, filtered_y_pred_sick_qa)
ci_sick_filtered_qa = compute_confidence_intervals(metrics_boot_sick_filtered_qa)
print("Healthy subjects (filtered):")
for metrica, (mean, lower, upper) in ci_healthy_filtered_qa.items():
    print(f"{metrica}: {mean:.3f} (IC 95%: {lower:.3f} - {upper:.3f})")
print("\nSick subjects (filtered):")
for metrica, (mean, lower, upper) in ci_sick_filtered_qa.items():
    print(f"{metrica}: {mean:.3f} (IC 95%: {lower:.3f} - {upper:.3f})")

Healthy subjects (filtered):
MAE_norm: 0.223 (IC 95%: 0.218 - 0.230)
MSE_norm: 0.077 (IC 95%: 0.074 - 0.081)
MAE_denorm: 7.030 (IC 95%: 6.853 - 7.235)
MSE_denorm: 76.466 (IC 95%: 73.086 - 80.497)
MAE_denorm_div7: 1.004 (IC 95%: 0.979 - 1.034)
MSE_denorm_div7: 10.924 (IC 95%: 10.441 - 11.500)
R2: 0.217 (IC 95%: 0.188 - 0.242)
P_value_R2: 0.000 (IC 95%: 0.000 - 0.000)
Pearson: 0.467 (IC 95%: 0.439 - 0.493)
Spearman: 0.467 (IC 95%: 0.441 - 0.494)
P_value_pearson: 0.000 (IC 95%: 0.000 - 0.000)
P_value_spearman: 0.000 (IC 95%: 0.000 - 0.000)
Slope: 0.237 (IC 95%: 0.221 - 0.252)

Sick subjects (filtered):
MAE_norm: 0.258 (IC 95%: 0.251 - 0.265)
MSE_norm: 0.100 (IC 95%: 0.095 - 0.105)
MAE_denorm: 8.120 (IC 95%: 7.908 - 8.343)
MSE_denorm: 99.063 (IC 95%: 94.511 - 103.952)
MAE_denorm_div7: 1.160 (IC 95%: 1.130 - 1.192)
MSE_denorm_div7: 14.152 (IC 95%: 13.502 - 14.850)
R2: 0.224 (IC 95%: 0.195 - 0.252)
P_value_R2: 0.000 (IC 95%: 0.000 - 0.000)
Pearson: 0.475 (IC 95%: 0.444 - 0.503)
Spearman: 0.4

In [36]:
df_healthy_qa = pd.DataFrame(ci_healthy_qa, index=['Media', 'IC 2.5%', 'IC 97.5%']).T
print(df_healthy_qa)
df_sick_qa = pd.DataFrame(ci_sick_qa, index=['Media', 'IC 2.5%', 'IC 97.5%']).T
print(df_sick_qa)
df_healthy_filtered_qa = pd.DataFrame(ci_healthy_filtered_qa, index=['Media', 'IC 2.5%', 'IC 97.5%']).T
print(df_healthy_filtered_qa)
df_sick_filtered_qa = pd.DataFrame(ci_sick_filtered_qa, index=['Media', 'IC 2.5%', 'IC 97.5%']).T
print(df_sick_filtered_qa)

                          Media        IC 2.5%       IC 97.5%
MAE_norm           2.570103e-01   2.505991e-01   2.636483e-01
MSE_norm           1.055011e-01   1.005020e-01   1.104476e-01
MAE_denorm         8.095824e+00   7.893871e+00   8.304921e+00
MSE_denorm         1.046835e+02   9.972313e+01   1.095916e+02
MAE_denorm_div7    1.156546e+00   1.127696e+00   1.186417e+00
MSE_denorm_div7    1.495478e+01   1.424616e+01   1.565595e+01
R2                 2.629411e-01   2.414061e-01   2.857121e-01
P_value_R2         1.110223e-16   1.110223e-16   1.110223e-16
Pearson            5.191408e-01   4.946123e-01   5.442192e-01
Spearman           5.158229e-01   4.907389e-01   5.421570e-01
P_value_pearson   3.984084e-202  3.418544e-269  1.838546e-215
P_value_spearman  1.148512e-201  8.930057e-267  1.270503e-211
Slope              2.283090e-01   2.155323e-01   2.415965e-01
                         Media       IC 2.5%      IC 97.5%
MAE_norm          5.163890e-01  5.029252e-01  5.305045e-01
MSE_norm      

ResNet with FMP

In [37]:
model_fmp = load_model(r'C:\Users\Utente\Desktop\Chiara\Codici\Cose nuove\Nuovi\Final_mean\Codes\Trained models\model_FMP_mean.keras')

In [38]:
train = pd.read_pickle(r"C:\Users\Utente\Desktop\Chiara\SUBSET\df_mean_train_FMP.pkl")
train['FHR'] = train['FHR'].apply(lambda x: x[0] if isinstance(x, list) else x)
train['FMP'] = train['FMP'].apply(lambda x: x[0] if isinstance(x, list) else x)

In [39]:
test = pd.read_pickle(r"C:\Users\Utente\Desktop\Chiara\SUBSET\df_mean_test_FMP.pkl")
test['FHR'] = test['FHR'].apply(lambda x: x[0] if isinstance(x, list) else x)
test['FMP'] = test['FMP'].apply(lambda x: x[0] if isinstance(x, list) else x)

In [40]:
test_sick = pd.read_pickle(r"C:\Users\Utente\Desktop\Chiara\SUBSET\df_mean_malati_FMP.pkl")
test_sick['FHR'] = test_sick['FHR'].apply(lambda x: x[0] if isinstance(x, list) else x)
test_sick['FMP'] = test_sick['FMP'].apply(lambda x: x[0] if isinstance(x, list) else x)

In [41]:
X_windows_train_FHR, y_windows_train= WindowSignals(train['FHR'], train['y'])
X_windows_test_FHR, y_windows_test = WindowSignals(test['FHR'], test['y'])
X_windows_test_sick_FHR, y_windows_test_sick = WindowSignals(test_sick['FHR'], test_sick['y'])

In [42]:
X_windows_train_FMP, y_windows_train= WindowSignals(train['FMP'], train['y'])
X_windows_test_FMP, y_windows_test = WindowSignals(test['FMP'], test['y'])
X_windows_test_sick_FMP, y_windows_test_sick = WindowSignals(test_sick['FHR'], test_sick['y'])

In [43]:
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(y_windows_train.reshape(-1, 1))

y_windows_train_scaled = scaler.transform(y_windows_train.reshape(-1, 1))
y_windows_test_scaled = scaler.transform(y_windows_test.reshape(-1, 1))
y_windows_test_sick_scaled = scaler.transform(y_windows_test_sick.reshape(-1, 1))
y_windows_train = y_windows_train_scaled
y_windows_test = y_windows_test_scaled
y_windows_test_sick = y_windows_test_sick_scaled

In [44]:
X_test = np.stack([X_windows_test_FHR, X_windows_test_FMP], axis=-1)
X_test_sick = np.stack([X_windows_test_sick_FHR, X_windows_test_sick_FMP], axis=-1) 
X_windows_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
X_windows_test_sick = tf.convert_to_tensor(X_test_sick, dtype=tf.float32)

In [45]:
y_windows_test = tf.convert_to_tensor(y_windows_test, dtype=tf.float32)
y_windows_test_sick = tf.convert_to_tensor(y_windows_test_sick, dtype=tf.float32)

In [46]:
#y_true_healthy = y_windows_test.flatten()
y_pred_healthy_fmp = model_fmp.predict(X_windows_test).flatten()
metrics_boot_healthy_fmp = bootstrap_metrics(y_true_healthy, y_pred_healthy_fmp)
ci_healthy_fmp = compute_confidence_intervals(metrics_boot_healthy_fmp)

#y_true_sick = y_windows_test_sick.flatten()
y_pred_sick_fmp = model_fmp.predict(X_windows_test_sick).flatten()
metrics_boot_sick_fmp = bootstrap_metrics(y_true_sick, y_pred_sick_fmp)
ci_sick_fmp = compute_confidence_intervals(metrics_boot_sick_fmp)

120/120 [==============================] - 6s 47ms/step


In [47]:
print("Healthy subjects:")
for metrica, (mean, lower, upper) in ci_healthy_fmp.items():
    print(f"{metrica}: {mean:.3f} (IC 95%: {lower:.3f} - {upper:.3f})")

print("\nSick subjects:")
for metrica, (mean, lower, upper) in ci_sick_fmp.items():
    print(f"{metrica}: {mean:.3f} (IC 95%: {lower:.3f} - {upper:.3f})")

Healthy subjects:
MAE_norm: 0.255 (IC 95%: 0.248 - 0.262)
MSE_norm: 0.105 (IC 95%: 0.101 - 0.110)
MAE_denorm: 8.033 (IC 95%: 7.824 - 8.245)
MSE_denorm: 104.522 (IC 95%: 99.907 - 109.407)
MAE_denorm_div7: 1.148 (IC 95%: 1.118 - 1.178)
MSE_denorm_div7: 14.932 (IC 95%: 14.272 - 15.630)
R2: 0.265 (IC 95%: 0.242 - 0.287)
P_value_R2: 0.000 (IC 95%: 0.000 - 0.000)
Pearson: 0.527 (IC 95%: 0.501 - 0.552)
Spearman: 0.519 (IC 95%: 0.493 - 0.544)
P_value_pearson: 0.000 (IC 95%: 0.000 - 0.000)
P_value_spearman: 0.000 (IC 95%: 0.000 - 0.000)
Slope: 0.232 (IC 95%: 0.219 - 0.244)

Sick subjects:
MAE_norm: 0.673 (IC 95%: 0.656 - 0.691)
MSE_norm: 0.737 (IC 95%: 0.697 - 0.782)
MAE_denorm: 21.196 (IC 95%: 20.657 - 21.752)
MSE_denorm: 731.191 (IC 95%: 692.001 - 775.883)
MAE_denorm_div7: 3.028 (IC 95%: 2.951 - 3.107)
MSE_denorm_div7: 104.456 (IC 95%: 98.857 - 110.840)
R2: -0.284 (IC 95%: -0.346 - -0.237)
P_value_R2: 1.000 (IC 95%: 1.000 - 1.000)
Pearson: -0.033 (IC 95%: -0.061 - -0.005)
Spearman: 0.027 (IC 

In [48]:
#filtered_y_true = np.array(y_true_healthy)[mask]
filtered_y_pred_fmp= np.array(y_pred_healthy_fmp)[mask]
#filtered_y_true_sick = np.array(y_true_sick)[mask_sick]
filtered_y_pred_sick_fmp = np.array(y_pred_sick_fmp)[mask_sick]

In [49]:
metrics_boot_healthy_filtered_fmp = bootstrap_metrics(filtered_y_true, filtered_y_pred_fmp)
ci_healthy_filtered_fmp = compute_confidence_intervals(metrics_boot_healthy_filtered_fmp)
metrics_boot_sick_filtered_fmp = bootstrap_metrics(filtered_y_true_sick, filtered_y_pred_sick_fmp)
ci_sick_filtered_fmp = compute_confidence_intervals(metrics_boot_sick_filtered_fmp)
print("Healthy subjects (filtered):")
for metrica, (mean, lower, upper) in ci_healthy_filtered_fmp.items():
    print(f"{metrica}: {mean:.3f} (IC 95%: {lower:.3f} - {upper:.3f})")
print("\nSick subjects (filtered):")
for metrica, (mean, lower, upper) in ci_sick_filtered_fmp.items():
    print(f"{metrica}: {mean:.3f} (IC 95%: {lower:.3f} - {upper:.3f})")

Healthy subjects (filtered):
MAE_norm: 0.222 (IC 95%: 0.216 - 0.228)
MSE_norm: 0.078 (IC 95%: 0.074 - 0.081)
MAE_denorm: 6.991 (IC 95%: 6.801 - 7.182)
MSE_denorm: 77.056 (IC 95%: 73.307 - 80.854)
MAE_denorm_div7: 0.999 (IC 95%: 0.972 - 1.026)
MSE_denorm_div7: 11.008 (IC 95%: 10.472 - 11.551)
R2: 0.213 (IC 95%: 0.185 - 0.241)
P_value_R2: 0.000 (IC 95%: 0.000 - 0.000)
Pearson: 0.470 (IC 95%: 0.443 - 0.498)
Spearman: 0.468 (IC 95%: 0.440 - 0.496)
P_value_pearson: 0.000 (IC 95%: 0.000 - 0.000)
P_value_spearman: 0.000 (IC 95%: 0.000 - 0.000)
Slope: 0.238 (IC 95%: 0.222 - 0.254)

Sick subjects (filtered):
MAE_norm: 0.399 (IC 95%: 0.387 - 0.412)
MSE_norm: 0.263 (IC 95%: 0.246 - 0.282)
MAE_denorm: 12.579 (IC 95%: 12.189 - 12.979)
MSE_denorm: 261.094 (IC 95%: 244.495 - 279.466)
MAE_denorm_div7: 1.797 (IC 95%: 1.741 - 1.854)
MSE_denorm_div7: 37.299 (IC 95%: 34.928 - 39.924)
R2: -1.046 (IC 95%: -1.195 - -0.911)
P_value_R2: 1.000 (IC 95%: 1.000 - 1.000)
Pearson: 0.006 (IC 95%: -0.034 - 0.047)
Spea

In [50]:
df_healthy_fmp = pd.DataFrame(ci_healthy_fmp, index=['Media', 'IC 2.5%', 'IC 97.5%']).T
print(df_healthy_fmp)
df_sick_fmp = pd.DataFrame(ci_sick_fmp, index=['Media', 'IC 2.5%', 'IC 97.5%']).T
print(df_sick_fmp)
df_healthy_filtered_fmp = pd.DataFrame(ci_healthy_filtered_fmp, index=['Media', 'IC 2.5%', 'IC 97.5%']).T
print(df_healthy_filtered_fmp)
df_sick_filtered_fmp = pd.DataFrame(ci_sick_filtered_fmp, index=['Media', 'IC 2.5%', 'IC 97.5%']).T
print(df_sick_filtered_fmp)

                          Media        IC 2.5%       IC 97.5%
MAE_norm           2.550285e-01   2.483697e-01   2.617386e-01
MSE_norm           1.053383e-01   1.006875e-01   1.102617e-01
MAE_denorm         8.033398e+00   7.823647e+00   8.244767e+00
MSE_denorm         1.045219e+02   9.990721e+01   1.094072e+02
MAE_denorm_div7    1.147628e+00   1.117664e+00   1.177824e+00
MSE_denorm_div7    1.493171e+01   1.427246e+01   1.562960e+01
R2                 2.648138e-01   2.418817e-01   2.867301e-01
P_value_R2         1.110223e-16   1.110223e-16   1.110223e-16
Pearson            5.270374e-01   5.009799e-01   5.518522e-01
Spearman           5.191237e-01   4.926003e-01   5.440910e-01
P_value_pearson   2.560207e-204  2.770588e-278  7.175608e-222
P_value_spearman  7.301774e-200  4.840962e-269  1.900080e-213
Slope              2.315540e-01   2.191740e-01   2.440322e-01
                       Media     IC 2.5%    IC 97.5%
MAE_norm            0.672901    0.655783    0.690548
MSE_norm            0.7369